In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from tabulate import tabulate

In [29]:

def read_cleaned_data(ticker):
    data = pd.read_csv(f'./Data/{ticker}.NS.csv')

    # Data Cleaning
    data.drop([0, 1], inplace=True)
    data.rename(columns={'Price': 'Date'}, inplace=True)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

    # Converting Object to Float
    data['Close'] = data['Close'].astype('Float64')
    data['High'] = data['High'].astype('Float64')
    data['Low'] = data['Low'].astype('Float64')
    data['Open'] = data['Open'].astype('Float64')
    data['Volume'] = data['Volume'].astype('Float64')

    return data

def goldencrossoversignalgenerator(ticker):
    # Fetching Data from csv file
    data = pd.read_csv(f'./Data/{ticker}.NS.csv')

    # Data Cleaning
    data.drop([0, 1], inplace=True)
    data.rename(columns={'Price': 'Date'}, inplace=True)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

    # Converting Object to Float
    data['Close'] = data['Close'].astype('Float64')
    data['High'] = data['High'].astype('Float64')
    data['Low'] = data['Low'].astype('Float64')
    data['Open'] = data['Open'].astype('Float64')
    data['Volume'] = data['Volume'].astype('Float64')
    #We generate signals based on closing price.So we act om the signals in the next Trading Session.That is,the next day.So we define the previous close
    data['Previous_Close'] = data['Close'].shift(1)
    # Calculating 20 SMA and 50 SMA
    data['20_SMA'] = data.Previous_Close.rolling(window=20).mean()
    data['50_SMA'] = data.Previous_Close.rolling(window=50).mean()

    # Generating Buy and Sell Signals
    data['Signal'] = 0
    data['Signal'] = np.where(data['20_SMA'] > data['50_SMA'], 1, 0)

    # Position where golden crossovers happen
    data['Position'] = data.Signal.diff()

    
    #recent = data.iloc[-300:]

    
    df_points = data[(data['Position'] == 1) | (data['Position'] == -1)].copy()
    f = lambda x: 'Buy' if x == 1 else 'Sell'
    df_points['Signal'] = df_points['Position'].apply(f)

    
    return df_points

In [30]:
data=goldencrossoversignalgenerator('RELIANCE')
data

,Close,High,Low,Open,Volume,Previous_Close,20_SMA,50_SMA,Signal,Position
Date,,,,,,,,,,
1996-03-11,6.452805,6.471883,6.314856,6.339803,631091821.0,6.43226,6.542693,5.944740,Buy,1.0
1996-04-05,6.122608,6.122608,6.122608,6.122608,0.0,6.122608,6.197525,6.232233,Sell,-1.0
1996-04-24,7.493297,7.57548,7.226202,7.252618,534899747.0,6.931226,6.413769,6.406490,Buy,1.0
1996-06-18,6.925356,7.073578,6.841706,7.073578,191150360.0,7.10733,7.077907,7.081532,Sell,-1.0
1996-08-13,6.141685,6.191582,6.069775,6.09032,196437024.0,6.106465,6.337456,6.336487,Buy,1.0
...,...,...,...,...,...,...,...,...,...,...
2024-06-14,1477.550049,1479.675049,1457.224976,1470.0,8157998.0,1465.25,1451.457495,1451.429497,Buy,1.0
2024-08-14,1461.849976,1472.0,1453.699951,1463.550049,6267466.0,1463.625,1498.647510,1505.612507,Sell,-1.0
2025-02-06,1281.550049,1288.0,1270.349976,1273.699951,9956001.0,1278.199951,1259.949988,1258.713000,Buy,1.0


In [31]:
#We must Remove biases in the data.Our first Signal cannot be a Sell. Our last signal cannot be a buy .
signal_df=data[(data.index >=data[data['Signal'] == 'Buy'].index[0]) & (data.index <=data[data['Signal'] == 'Sell'].index[-1])]
signal_df

,Close,High,Low,Open,Volume,Previous_Close,20_SMA,50_SMA,Signal,Position
Date,,,,,,,,,,
1996-03-11,6.452805,6.471883,6.314856,6.339803,631091821.0,6.43226,6.542693,5.944740,Buy,1.0
1996-04-05,6.122608,6.122608,6.122608,6.122608,0.0,6.122608,6.197525,6.232233,Sell,-1.0
1996-04-24,7.493297,7.57548,7.226202,7.252618,534899747.0,6.931226,6.413769,6.406490,Buy,1.0
1996-06-18,6.925356,7.073578,6.841706,7.073578,191150360.0,7.10733,7.077907,7.081532,Sell,-1.0
1996-08-13,6.141685,6.191582,6.069775,6.09032,196437024.0,6.106465,6.337456,6.336487,Buy,1.0
...,...,...,...,...,...,...,...,...,...,...
2024-05-03,1434.0,1474.949951,1416.150024,1471.0,17226958.0,1466.550049,1467.169995,1467.542996,Sell,-1.0
2024-06-14,1477.550049,1479.675049,1457.224976,1470.0,8157998.0,1465.25,1451.457495,1451.429497,Buy,1.0
2024-08-14,1461.849976,1472.0,1453.699951,1463.550049,6267466.0,1463.625,1498.647510,1505.612507,Sell,-1.0


In [32]:
class Backtest:
    def __init__(self):
        self.columns=['Stock Name','Trade','Entry Time','Entry Price','Exit Time','Exit Price','Quantity','Position Size','Gross PNL','Charges','Net PNL','%PNL','Holding Period']
        self.backtesting_df=pd.DataFrame(columns=self.columns)

    def Buy(self,stock_name,entry_time,entry_price,qty):
        self.trade_log = dict(zip(self.columns, [None] * len(self.columns)))
        self.trade_log['Trade'] = 'Open Long'
        self.trade_log['Stock Name'] = stock_name
        self.trade_log['Entry Time'] = entry_time
        self.trade_log['Entry Price'] = round(entry_price,3) 
        self.trade_log['Quantity'] = qty
        self.trade_log['Position Size'] = round(self.trade_log['Quantity'] * entry_price, 3)
        
    def Sell(self,stock_name,exit_time,exit_price,charges):
        self.trade_log['Trade']='Closed Long'
        self.trade_log['Stock Name'] = stock_name
        self.trade_log['Exit Time'] = exit_time
        self.trade_log['Exit Price'] = round(exit_price,3) 
        self.trade_log['Gross PNL']=round((self.trade_log['Exit Price']-self.trade_log['Entry Price'])*self.trade_log['Quantity'],3)
        self.trade_log['Charges']=round(charges,3)
        self.trade_log['Net PNL']=round(self.trade_log['Gross PNL']-self.trade_log['Charges'],3)
        self.trade_log['%PNL']=round(self.trade_log['Net PNL']/self.trade_log['Position Size']*100,3)
        self.trade_log['Holding Period']=self.trade_log['Exit Time']-self.trade_log['Entry Time']
        self.backtesting_df=pd.concat([self.backtesting_df,pd.DataFrame([self.trade_log])],ignore_index=True)

    def stats(self):
        trades_df=self.backtesting_df
        columns=['Stocks Traded','Total Number of Trades','Net PNL','Winning Trades','Losing Trades','Total Charges','Win Ratio','Risk Reward Ratio']
        stocks_traded=len(trades_df['Stock Name'].unique())
        total_number_of_trades=len(trades_df.index)
        pnl=round(trades_df['Net PNL'].sum(),2)
        total_charges=round(trades_df['Charges'].sum(),2)
        losing_trades=len(trades_df[trades_df['Net PNL']<=0].index)
        winning_trades=len(trades_df[trades_df['Net PNL']>0].index)
        win_ratio=str(round(winning_trades/total_number_of_trades,2)*100)+'%'
        total_profit=trades_df[trades_df['Net PNL']>0]['Net PNL'].sum()
        total_loss=trades_df[trades_df['Net PNL']<=0]['Net PNL'].sum()
        risk_reward_ratio=f'1:{-1*round((total_profit/winning_trades)/(total_loss/losing_trades),2)}'

        data_values=[stocks_traded,total_number_of_trades,pnl,winning_trades,losing_trades,total_charges,win_ratio,risk_reward_ratio]
        data=list(zip(columns,data_values))

        print(tabulate(data,['Parameters','Values'],tablefmt='fancy_grid'))

        
        

In [33]:
test=Backtest()
capital=1000000 #Capital for each stock
stocks_list=['RELIANCE','CAMS','HCLTECH','HAL','LT']
portfolio_value=[capital*len(stocks_list)]
for stock in stocks_list:
    data=goldencrossoversignalgenerator(stock)
    signal_df=data[(data.index >=data[data['Signal'] == 'Buy'].index[0]) & (data.index <=data[data['Signal'] == 'Sell'].index[-1])]
    for index,data in signal_df.iterrows():
        if data['Signal']=='Buy':
            qty=capital//data['Open']
            test.Buy(stock,index,data['Open'],qty)
        else:
            #Considering 0.5% charges
            charges=qty=(capital//data['Open'])*data['Open']*0.005
            test.Sell(stock,index,data['Open'],charges)
            portfolio_value.append(portfolio_value[-1]+test.backtesting_df.iloc[-1]['Net PNL'])

In [34]:

test.backtesting_df

,Stock Name,Trade,Entry Time,Entry Price,Exit Time,Exit Price,Quantity,Position Size,Gross PNL,Charges,Net PNL,%PNL,Holding Period
0,RELIANCE,Closed Long,1996-03-11,6.340,1996-04-05,6.123,157733.0,999996.205,-34228.061,4999.997,-39228.058,-3.923,25 days
1,RELIANCE,Closed Long,1996-04-24,7.253,1996-06-18,7.074,137881.0,999998.268,-24680.699,4999.994,-29680.693,-2.968,55 days
2,RELIANCE,Closed Long,1996-08-13,6.090,1996-08-16,6.061,164194.0,999994.062,-4761.626,4999.997,-9761.623,-0.976,3 days
3,RELIANCE,Closed Long,1996-08-30,6.196,1996-09-17,5.666,161394.0,999994.841,-85538.820,4999.996,-90538.816,-9.054,18 days
4,RELIANCE,Closed Long,1996-11-12,5.384,1996-11-21,5.358,185720.0,999996.327,-4828.720,4999.992,-9828.712,-0.983,9 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,LT,Closed Long,2024-03-15,3536.950,2024-05-10,3225.919,282.0,997420.021,-87710.742,4984.045,-92694.787,-9.293,56 days
239,LT,Closed Long,2024-06-18,3645.084,2024-08-30,3683.992,274.0,998752.972,10660.792,4991.809,5668.983,0.568,73 days
240,LT,Closed Long,2024-09-17,3634.107,2024-10-18,3536.963,275.0,999379.437,-26714.600,4987.118,-31701.718,-3.172,31 days
241,LT,Closed Long,2024-11-29,3601.511,2025-01-13,3473.158,277.0,997618.606,-35553.781,4983.982,-40537.763,-4.063,45 days


In [35]:
test.stats()

╒════════════════════════╤═════════════╕
│ Parameters             │ Values      │
╞════════════════════════╪═════════════╡
│ Stocks Traded          │ 5           │
├────────────────────────┼─────────────┤
│ Total Number of Trades │ 243         │
├────────────────────────┼─────────────┤
│ Net PNL                │ 16724952.29 │
├────────────────────────┼─────────────┤
│ Winning Trades         │ 104         │
├────────────────────────┼─────────────┤
│ Losing Trades          │ 139         │
├────────────────────────┼─────────────┤
│ Total Charges          │ 1214573.49  │
├────────────────────────┼─────────────┤
│ Win Ratio              │ 43.0%       │
├────────────────────────┼─────────────┤
│ Risk Reward Ratio      │ 1:3.81      │
╘════════════════════════╧═════════════╛


In [36]:
#Returns and comparison
trading_returns=test.backtesting_df['Net PNL'].sum()/(capital*len(stocks_list))*100
returns=0
for stock in stocks_list:
    diff=read_cleaned_data(stock)['Close'].iloc[-1]-read_cleaned_data(stock)['Close'].iloc[0]
    number_bought=capital//read_cleaned_data(stock)['Close'].iloc[0]
    returns+=diff*(number_bought)-capital*0.005

buy_and_hold_returns=returns/(capital*len(stocks_list))*100
print(f'Trading Returns: {trading_returns}')
print(f'Buy and Hold Returns: {buy_and_hold_returns}')


Trading Returns: 334.49904572
Buy and Hold Returns: 9655.192986545877


In [38]:
portfolio_value

[5000000,
 4960771.942,
 4931091.249,
 4921329.626,
 4830790.8100000005,
 4820962.098,
 5092585.506,
 5244241.197,
 4997563.472,
 4917456.3100000005,
 4956078.983000001,
 4902374.638000001,
 5056668.5320000015,
 4845549.500000002,
 4886691.560000001,
 5085933.709000002,
 4986532.630000002,
 5119219.786000002,
 5072377.943000002,
 4910111.8840000015,
 5076629.792000001,
 5011238.3900000015,
 5176509.154000002,
 5135979.696000002,
 5145596.304000002,
 5080035.826000002,
 5251447.904000002,
 5220156.790000002,
 5170787.970000002,
 5880076.028000002,
 5820955.524000002,
 5916780.022000002,
 5933026.455000002,
 6151590.415000002,
 6956623.121000002,
 6969766.619000002,
 7163649.035000002,
 7152843.914000002,
 7835204.517000002,
 7665611.7420000015,
 7468216.683000001,
 7523018.165000001,
 7481374.245000001,
 7755910.925000001,
 7739415.553,
 7616874.203000001,
 7594034.109000001,
 7552161.156000001,
 7517657.054000001,
 7407102.211000001,
 7322488.053000001,
 7314051.947000002,
 7226312.977